# Applied Capstone Project - Week 5

# Capstone Project - The Battle of Neighborhoods

### 1. Installing and Importing Python Libraries and Dependencies

In [3]:
import pandas as pd
import numpy as np
import requests
import geopy
import geocoder
import folium
import json
import xml
import matplotlib.cm as cm
import matplotlib.colors as colors
import warnings
warnings.filterwarnings("ignore")

from geopy.geocoders import Nominatim
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print('Libraries imported.')

Libraries imported.


### 2. Data Extraction and Cleaning

Webscraping - retrieving the URL and creating a Beautiful soup object

In [4]:
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(data, 'lxml')

Create lists to store the data. Use split ,strip and replace functions for getting Borough and Neighborhood information.

In [5]:
table_contents = []
table = soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

Transform the data into pandas dataframe

In [6]:
df = pd.DataFrame(table_contents)
df['Borough'] = df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

Remove rows with 'Not assigned' in Borough column

In [7]:
df = df[df.Borough != 'Not assigned'].reset_index(drop = True)

Group multiple neighborhood with the same postal code

In [9]:
df_group = df.groupby(['PostalCode', 'Borough'], as_index=False).agg(lambda x: ", ".join(x))
df_group['Neighborhood'] = df_group['Neighborhood'].str.replace('/', ',')
df_group

PostalCode                 Borough  \
0          M1B             Scarborough   
1          M1C             Scarborough   
2          M1E             Scarborough   
3          M1G             Scarborough   
4          M1H             Scarborough   
5          M1J             Scarborough   
6          M1K             Scarborough   
7          M1L             Scarborough   
8          M1M             Scarborough   
9          M1N             Scarborough   
10         M1P             Scarborough   
11         M1R             Scarborough   
12         M1S             Scarborough   
13         M1T             Scarborough   
14         M1V             Scarborough   
15         M1W             Scarborough   
16         M1X             Scarborough   
17         M2H              North York   
18         M2J              North York   
19         M2K              North York   
20         M2L              North York   
21         M2M              North York   
22         M2N              North York   
23         M2P              North York   
24         M2R              North York   
25         M3A              North York   
26         M3B              North York   
27         M3C              North York   
28         M3H              North York   
29         M3J              North York   
30         M3K              North York   
31         M3L              North York   
32         M3M              North York   
33         M3N              North York   
34         M4A              North York   
35         M4B               East York   
36         M4C               East York   
37         M4E            East Toronto   
38         M4G               East York   
39         M4H               East York   
40         M4J  East York/East Toronto   
41         M4K            East Toronto   
42         M4L            East Toronto   
43         M4M            East Toronto   
44         M4N         Central Toronto   
45         M4P         Central Toronto   
46         M4R         Central Toronto   
47         M4S         Central Toronto   
48         M4T         Central Toronto   
49         M4V         Central Toronto   
50         M4W        Downtown Toronto   
51         M4X        Downtown Toronto   
52         M4Y        Downtown Toronto   
53         M5A        Downtown Toronto   
54         M5B        Downtown Toronto   
55         M5C        Downtown Toronto   
56         M5E        Downtown Toronto   
57         M5G        Downtown Toronto   
58         M5H        Downtown Toronto   
59         M5J        Downtown Toronto   
60         M5K        Downtown Toronto   
61         M5L        Downtown Toronto   
62         M5M              North York   
63         M5N         Central Toronto   
64         M5P         Central Toronto   
65         M5R         Central Toronto   
66         M5S        Downtown Toronto   
67         M5T        Downtown Toronto   
68         M5V        Downtown Toronto   
69         M5W  Downtown Toronto Stn A   
70         M5X        Downtown Toronto   
71         M6A              North York   
72         M6B              North York   
73         M6C                    York   
74         M6E                    York   
75         M6G        Downtown Toronto   
76         M6H            West Toronto   
77         M6J            West Toronto   
78         M6K            West Toronto   
79         M6L              North York   
80         M6M                    York   
81         M6N                    York   
82         M6P            West Toronto   
83         M6R            West Toronto   
84         M6S            West Toronto   
85         M7A            Queen's Park   
86         M7R             Mississauga   
87         M7Y   East Toronto Business   
88         M8V               Etobicoke   
89         M8W               Etobicoke   
90         M8X               Etobicoke   
91         M8Y               Etobicoke   
92         M8Z               Etobicoke   
93         M9A               Etobicoke   
94         M9B

Read the cvs file that contains the geographical coordinates of the neighboorhods

In [17]:
coordinates = pd.read_csv('Geospatial_Coordinates.csv')
coordinates.rename(columns = {"Postal Code": "PostalCode"}, inplace = True)

Merge two dataframes that contain borough-neighborhoods and latitude-longitude

In [18]:
neighborhoods = pd.merge(df_group, coordinates, on='PostalCode')
neighborhoods.head()

PostalCode      Borough                            Neighborhood   Latitude  \
0        M1B  Scarborough                          Malvern, Rouge  43.806686   
1        M1C  Scarborough  Rouge Hill, Port Union, Highland Creek  43.784535   
2        M1E  Scarborough       Guildwood, Morningside, West Hill  43.763573   
3        M1G  Scarborough                                  Woburn  43.770992   
4        M1H  Scarborough                               Cedarbrae  43.773136   

   Longitude  
0 -79.194353  
1 -79.160497  
2 -79.188711  
3 -79.216917  
4 -79.239476

In [19]:
neighborhoods_ny = neighborhoods[neighborhoods['Borough']=='North York'].reset_index(drop = True)

In [20]:
neighborhoods_ny

PostalCode     Borough                                     Neighborhood  \
0         M2H  North York                                Hillcrest Village   
1         M2J  North York                     Fairview, Henry Farm, Oriole   
2         M2K  North York                                  Bayview Village   
3         M2L  North York                         York Mills, Silver Hills   
4         M2M  North York                          Willowdale, Newtonbrook   
5         M2N  North York                                 Willowdale South   
6         M2P  North York                                  York Mills West   
7         M2R  North York                                  Willowdale West   
8         M3A  North York                                        Parkwoods   
9         M3B  North York                                  Don Mills North   
10        M3C  North York                                  Don Mills South   
11        M3H  North York  Bathurst Manor, Wilson Heights, Downsview North   
12        M3J  North York                  Northwood Park, York University   
13        M3K  North York                                   Downsview East   
14        M3L  North York                                   Downsview West   
15        M3M  North York                                Downsview Central   
16        M3N  North York                              Downsview Northwest   
17        M4A  North York                                 Victoria Village   
18        M5M  North York                Bedford Park, Lawrence Manor East   
19        M6A  North York                 Lawrence Manor, Lawrence Heights   
20        M6B  North York                                        Glencairn   
21        M6L  North York         North Park, Maple Leaf Park, Upwood Park   
22        M9L  North York                                    Humber Summit   
23        M9M  North York                                 Humberlea, Emery   

     Latitude  Longitude  
0   43.803762 -79.363452  
1   43.778517 -79.346556  
2   43.786947 -79.385975  
3   43.757490 -79.374714  
4   43.789053 -79.408493  
5   43.770120 -79.408493  
6   43.752758 -79.400049  
7   43.782736 -79.442259  
8   43.753259 -79.329656  
9   43.745906 -79.352188  
10  43.725900 -79.340923  
11  43.754328 -79.442259  
12  43.767980 -79.487262  
13  43.737473 -79.464763  
14  43.739015 -79.506944  
15  43.728496 -79.495697  
16  43.761631 -79.520999  
17  43.725882 -79.315572  
18  43.733283 -79.419750  
19  43.718518 -79.464763  
20  43.709577 -79.445073  
21  43.713756 -79.490074  
22  43.756303 -79.565963  
23  43.724766 -79.532242

### 3. Map of Scarborough

Use geopy library to get the latitude and longitude values of North York

In [21]:
address = 'North York, Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of North York are 43.7543263, -79.44911696639593.


Create a map of North York with neighborhoods superimposed on top.

In [22]:
# create map of North York using latitude and longitude values
map_ny = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html = False).add_to(map_ny)  
    
map_ny

In [29]:
address = 'North York,Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude_n1 = location.latitude
longitude_n1 = location.longitude
print('The Geograpical Co-ordinate of Neighborhood_1 are {}, {}.'.format(latitude, longitude))

The Geograpical Co-ordinate of Neighborhood_1 are 43.7543263, -79.44911696639593.


In [30]:
# @hiddel_cell
CLIENT_ID = 'HZOY123SVG0VENO1QRCM5UGRYHRKS41C0JPG03WHI1QGB0MU' # my Foursquare ID
CLIENT_SECRET = '5WJVCZZ5QZGGHLHO0H4I3NYQ1N2CTOEJEVRUXVEHGUYEGM1I' # my Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: '+CLIENT_ID)
print('CLIENT_SECRET: '+CLIENT_SECRET)

Your credentails:
CLIENT_ID: HZOY123SVG0VENO1QRCM5UGRYHRKS41C0JPG03WHI1QGB0MU
CLIENT_SECRET: 5WJVCZZ5QZGGHLHO0H4I3NYQ1N2CTOEJEVRUXVEHGUYEGM1I


In [31]:
radius = 700 
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude_n1, 
   longitude_n1, 
    radius, 
   LIMIT)
results = requests.get(url).json()

In [32]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)
nearby_venues.columns

Index(['referralId', 'reasons.count', 'reasons.items', 'venue.id',
       'venue.name', 'venue.location.address', 'venue.location.crossStreet',
       'venue.location.lat', 'venue.location.lng',
       'venue.location.labeledLatLngs', 'venue.location.distance',
       'venue.location.postalCode', 'venue.location.cc',
       'venue.location.neighborhood', 'venue.location.city',
       'venue.location.state', 'venue.location.country',
       'venue.location.formattedAddress', 'venue.categories',
       'venue.photos.count', 'venue.photos.groups'],
      dtype='object')

In [33]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

### 4. Nearby Venues/Locations

In [34]:
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues.head()

venue.name                                   venue.categories  \
0               Grill Gate  [{'id': '4bf58dd8d48988d1c0941735', 'name': 'M...   
1            Wolfie's Deli  [{'id': '4bf58dd8d48988d146941735', 'name': 'D...   
2               Bagel Plus  [{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...   
3  Orly Restaurant & Grill  [{'id': '4bf58dd8d48988d115941735', 'name': 'M...   
4              Tim Hortons  [{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...   

   venue.location.lat  venue.location.lng  
0           43.753123          -79.451690  
1           43.754875          -79.442438  
2           43.755395          -79.440686  
3           43.754493          -79.443507  
4           43.754767          -79.443250

### 5. Categories of Nearby Venues/Locations

In [35]:
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(5)

name                 categories        lat        lng
0               Grill Gate   Mediterranean Restaurant  43.753123 -79.451690
1            Wolfie's Deli              Deli / Bodega  43.754875 -79.442438
2               Bagel Plus                 Restaurant  43.755395 -79.440686
3  Orly Restaurant & Grill  Middle Eastern Restaurant  43.754493 -79.443507
4              Tim Hortons                Coffee Shop  43.754767 -79.443250

In [36]:
# Top 10 Categories
a=pd.Series(nearby_venues.categories)
a.value_counts()[:10]

Pizza Place                  2
Fried Chicken Joint          1
Deli / Bodega                1
Convenience Store            1
Sushi Restaurant             1
Middle Eastern Restaurant    1
Mediterranean Restaurant     1
Coffee Shop                  1
American Restaurant          1
Restaurant                   1
Name: categories, dtype: int64

In [62]:
def getNearbyVenues(names, latitudes, longitudes, radius=700):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # making GET request
        venue_results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in venue_results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [63]:
# Nearby Venues
northyork_venues = getNearbyVenues(names = neighborhoods_ny['Neighborhood'],
                                   latitudes = neighborhoods_ny['Latitude'],
                                   longitudes = neighborhoods_ny['Longitude'])

Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Don Mills South
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Bedford Park, Lawrence Manor East
Lawrence Manor, Lawrence Heights
Glencairn
North Park, Maple Leaf Park, Upwood Park
Humber Summit
Humberlea, Emery


In [64]:
print('There are {} Uniques Categories.'.format(len(northyork_venues['Venue Category'].unique())))
northyork_venues.groupby('Neighborhood').count().head()

There are 117 Uniques Categories.


Neighborhood Latitude  \
Neighborhood                                                             
Bathurst Manor, Wilson Heights, Downsview North                     25   
Bayview Village                                                      8   
Bedford Park, Lawrence Manor East                                   31   
Don Mills North                                                      5   
Don Mills South                                                     23   

                                                 Neighborhood Longitude  \
Neighborhood                                                              
Bathurst Manor, Wilson Heights, Downsview North                      25   
Bayview Village                                                       8   
Bedford Park, Lawrence Manor East                                    31   
Don Mills North                                                       5   
Don Mills South                                                      23   

                                                 Venue  Venue Latitude  \
Neighborhood                                                             
Bathurst Manor, Wilson Heights, Downsview North     25              25   
Bayview Village                                      8               8   
Bedford Park, Lawrence Manor East                   31              31   
Don Mills North                                      5               5   
Don Mills South                                     23              23   

                                                 Venue Longitude  \
Neighborhood                                                       
Bathurst Manor, Wilson Heights, Downsview North               25   
Bayview Village                                                8   
Bedford Park, Lawrence Manor East                             31   
Don Mills North                                                5   
Don Mills South                                               23   

                                                 Venue Category  
Neighborhood                                                     
Bathurst Manor, Wilson Heights, Downsview North              25  
Bayview Village                                               8  
Bedford Park, Lawrence Manor East                            31  
Don Mills North                                               5  
Don Mills South                                              23

### One Hot Encoding of Features

In [77]:
# one hot encoding
northyork_onehot = pd.get_dummies(northyork_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
northyork_onehot['Neighborhood'] = northyork_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [northyork_onehot.columns[-1]] + list(northyork_onehot.columns[:-1])
northyork_onehot = northyork_onehot[fixed_columns]
northyork_grouped = northyork_onehot.groupby('Neighborhood').mean().reset_index()
northyork_onehot.head(5)

Neighborhood  Accessories Store  Airport  American Restaurant  \
0  Hillcrest Village                  0        0                    0   
1  Hillcrest Village                  0        0                    0   
2  Hillcrest Village                  0        0                    0   
3  Hillcrest Village                  0        0                    0   
4  Hillcrest Village                  0        0                    0   

   Art Gallery  Arts & Crafts Store  Asian Restaurant  Athletics & Sports  \
0            0                    0                 0                   0   
1            0                    0                 0                   0   
2            0                    0                 0                   0   
3            0                    0                 0                   0   
4            0                    0                 0                   0   

   Bagel Shop  Bakery  Bank  Bar  Baseball Field  Beer Store  Bike Shop  \
0           0       1     0    0               0           0          0   
1           0       0     0    0               0           0          0   
2           0       0     0    0               0           0          0   
3           0       0     0    0               0           0          0   
4           0       0     0    0               0           0          0   

   Boutique  Bowling Alley  Bridal Shop  Bubble Tea Shop  Burger Joint  \
0         0              0            0                0             0   
1         0              0            0                0             0   
2         0              0            0                0             0   
3         0              0            0                0             0   
4         0              0            0                0             0   

   Burrito Place  Bus Line  Business Service  Butcher  Café  \
0              0         0                 0        0     0   
1              0         0                 0        0     0   
2              0         0                 0        0     0   
3              0         0                 0        0     0   
4              0         0                 0        0     0   

   Caribbean Restaurant  Cheese Shop  Chinese Restaurant  Chocolate Shop  \
0                     0            0                   0               0   
1                     0            0                   0               0   
2                     0            0                   0               0   
3                     0            0                   0               0   
4                     0            0                   0               0   

   Clothing Store  Coffee Shop  Comfort Food Restaurant  Community Center  \
0               0            0                        0                 0   
1               0            0                        0                 0   
2               0            0                        0                 0   
3               0            0                        0                 0   
4               0            0                        0                 0   

   Construction & Landscaping  Convenience Store  Cosmetics Shop  \
0                           0                  0               0   
1                           0                  0               0   
2                           0                  0               0   
3                           0                  0               0   
4                           0                  0               0   

   Cupcake Shop  Dance Studio  Deli / Bodega  Department Store  Dessert Shop  \
0             0             0              0                 0             0   
1             0             0              0                 0             0   
2             0             0              0                 0             0   
3             0             0              0                 0             0   
4             0             0              0                 0             0   

   Dim Sum Restaurant

In [78]:
num_top_venues = 5
for hood in northyork_grouped['Neighborhood']:
    print("---- "+hood+" ----")
    temp = northyork_grouped[northyork_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---- Bathurst Manor, Wilson Heights, Downsview North ----
           venue  freq
0           Park  0.08
1    Coffee Shop  0.08
2           Bank  0.08
3    Bridal Shop  0.04
4  Deli / Bodega  0.04


---- Bayview Village ----
                 venue  freq
0                 Bank  0.25
1           Playground  0.12
2        Grocery Store  0.12
3                 Café  0.12
4  Japanese Restaurant  0.12


---- Bedford Park, Lawrence Manor East ----
                venue  freq
0         Coffee Shop  0.10
1  Italian Restaurant  0.10
2      Sandwich Place  0.06
3    Sushi Restaurant  0.03
4            Boutique  0.03


---- Don Mills North ----
                  venue  freq
0   Japanese Restaurant   0.4
1                   Gym   0.2
2  Caribbean Restaurant   0.2
3                  Café   0.2
4     Accessories Store   0.0


---- Don Mills South ----
                venue  freq
0         Coffee Shop  0.09
1          Restaurant  0.09
2                 Gym  0.09
3  Italian Restaurant  0.04
4         Su

In [79]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Most Common venues near neighborhood

In [80]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = northyork_grouped['Neighborhood']

for ind in np.arange(northyork_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(northyork_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhood 1st Most Common Venue  \
0  Bathurst Manor, Wilson Heights, Downsview North                  Park   
1                                  Bayview Village                  Bank   
2                Bedford Park, Lawrence Manor East           Coffee Shop   
3                                  Don Mills North   Japanese Restaurant   
4                                  Don Mills South           Coffee Shop   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0           Coffee Shop                  Bank           Bridal Shop   
1            Playground         Grocery Store                  Café   
2    Italian Restaurant        Sandwich Place      Sushi Restaurant   
3                   Gym  Caribbean Restaurant                  Café   
4            Restaurant                   Gym    Italian Restaurant   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0         Deli / Bodega                 Diner              Pharmacy   
1   Japanese Restaurant    Chinese Restaurant          Intersection   
2              Boutique            Restaurant                   Pub   
3     Accessories Store  Medical Supply Store                  Park   
4           Supermarket          Dance Studio        Discount Store   

  8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
0           Pizza Place     Convenience Store       Community Center  
1             Nightclub         Metro Station   Outdoor Supply Store  
2                  Café               Butcher           Cupcake Shop  
3  Outdoor Supply Store             Nightclub          Moving Target  
4             Nightclub          Intersection         Clothing Store

### K-Means Clustering Approach

In [81]:
# Using K-Means to cluster neighborhood into 3 clusters
northyork_grouped_clustering = northyork_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters = 3, random_state = 0).fit(northyork_grouped_clustering)
kmeans.labels_

array([2, 2, 2, 1, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 0, 0, 2, 2, 0,
       0])

In [82]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

northyork_merged = neighborhoods_ny.iloc[:,:]

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
northyork_merged = northyork_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

northyork_merged.dropna(inplace = True)
northyork_merged.head()# check the last columns!

PostalCode     Borough                  Neighborhood   Latitude  Longitude  \
0        M2H  North York             Hillcrest Village  43.803762 -79.363452   
1        M2J  North York  Fairview, Henry Farm, Oriole  43.778517 -79.346556   
2        M2K  North York               Bayview Village  43.786947 -79.385975   
4        M2M  North York       Willowdale, Newtonbrook  43.789053 -79.408493   
5        M2N  North York              Willowdale South  43.770120 -79.408493   

   Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
0             2.0  Fast Food Restaurant    Chinese Restaurant   
1             2.0        Clothing Store           Coffee Shop   
2             2.0                  Bank            Playground   
4             0.0                  Park                 Trail   
5             2.0           Coffee Shop     Korean Restaurant   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
0        Sandwich Place            Restaurant          Tennis Court   
1  Fast Food Restaurant        Baseball Field         Women's Store   
2         Grocery Store                  Café   Japanese Restaurant   
4           Pizza Place           Coffee Shop        Ice Cream Shop   
5      Ramen Restaurant           Pizza Place          Dessert Shop   

  6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
0                Bakery              Pharmacy     Mobile Phone Shop   
1             Juice Bar   Japanese Restaurant     Mobile Phone Shop   
2    Chinese Restaurant          Intersection             Nightclub   
4                 Hotel          Home Service  Outdoor Supply Store   
5                  Café         Shopping Mall  Fast Food Restaurant   

  9th Most Common Venue     10th Most Common Venue  
0         Movie Theater             Massage Studio  
1                  Park                 Food Court  
2         Metro Station       Outdoor Supply Store  
4             Nightclub              Moving Target  
5      Sushi Restaurant  Middle Eastern Restaurant

### Map of Clusters

In [83]:
kclusters = 10

In [85]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
colors_array = cm.rainbow(np.linspace(0, 1, kclusters))
rainbow = [colors.rgb2hex(i) for i in colors_array]
print(rainbow)

# add markers to the map
markers_colors = []
for lat, lon, nei , cluster in zip(northyork_merged['Latitude'], 
                                   northyork_merged['Longitude'], 
                                   northyork_merged['Neighborhood'], 
                                   northyork_merged['Cluster Labels']):
    label = folium.Popup(str(nei) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius = 5,
        popup = label,
        color = rainbow[int(cluster)-1],
        fill = True,
        fill_color = rainbow[int(cluster)-1],
        fill_opacity = 0.7).add_to(map_clusters)
       
map_clusters

['#8000ff', '#4856fb', '#10a2f0', '#2adddd', '#62fbc4', '#9cfba4', '#d4dd80', '#ffa256', '#ff562c', '#ff0000']


In [86]:
df1 = northyork_merged.loc[northyork_merged['Cluster Labels'] == 0,northyork_merged.columns[[2] + list(range(5, northyork_merged.shape[1]))]]
df2 = northyork_merged.loc[northyork_merged['Cluster Labels'] == 1,northyork_merged.columns[[2] + list(range(5, northyork_merged.shape[1]))]]
df3 = northyork_merged.loc[northyork_merged['Cluster Labels'] == 2,northyork_merged.columns[[2] + list(range(5, northyork_merged.shape[1]))]]

In [90]:
northyork_Avg_HousingPrice=pd.DataFrame({"Neighborhood":neighborhoods_ny["Neighborhood"],
                                      "Average_Housing_Price":[335000, 286600, 175000, 225900, 219400, 438500,
                                                               573900, 225000, 370500, 370500, 433500, 279200,
                                                               279200, 225000, 370500, 255400, 433500, 433500,
                                                               435000, 289500, 265000, 285900, 239400, 589900]})

In [91]:
northyork_Avg_HousingPrice.set_index('Neighborhood',inplace=True,drop=True)

In [92]:
northyork_Avg_HousingPrice.plot(kind='bar',figsize=(24,18),alpha=0.75)